In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

# Qiskit runtime primitives on local



## Estimator 

In [2]:
from qiskit.primitives import Estimator
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

circuit = RealAmplitudes(num_qubits=2, reps=2)

observable = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])

theta = [0, 1, 1, 2, 3, 5]


estimator = Estimator()


job = estimator.run([circuit], [observable], [theta])
job_result = job.result() # It will block until the job finishes.
print(f"The primitive-job finished with result {job_result}")

The primitive-job finished with result EstimatorResult(values=array([1.55555728]), metadata=[{}])


**Multiple circuites and observables** 

In [3]:

from qiskit.primitives import Estimator
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

psi1 = RealAmplitudes(num_qubits=2, reps=2)
psi2 = RealAmplitudes(num_qubits=2, reps=3)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
theta3 = [1, 2, 3, 4, 5, 6]

estimator = Estimator()

# calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
job = estimator.run([psi1], [H1], [theta1])
job_result = job.result() # It will block until the job finishes.
print(f"The primitive-job finished with result {job_result}")

# calculate [ <psi1(theta1)|H1|psi1(theta1)>,
#             <psi2(theta2)|H2|psi2(theta2)>,
#             <psi1(theta3)|H3|psi1(theta3)> ]
job2 = estimator.run([psi1, psi2, psi1], [H1, H2, H3], [theta1, theta2, theta3])
job_result = job2.result()
print(f"The primitive-job finished with result {job_result}")

The primitive-job finished with result EstimatorResult(values=array([1.55555728]), metadata=[{}])
The primitive-job finished with result EstimatorResult(values=array([ 1.55555728,  0.17849238, -1.08766318]), metadata=[{}, {}, {}])


## IBM Quantum setup

In [4]:
# Save IBM Cloud otherwise we get: AccountNotFoundError: 'No default ibm_cloud account saved.'
from qiskit_ibm_runtime import QiskitRuntimeService

# Save an IBM Quantum account.
QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_IBM_QUANTUM_TOKEN>")


AccountAlreadyExistsError: 'Named account (default-ibm-quantum) already exists. Set overwrite=True to overwrite.'

## IBM Cloud setup

In [5]:
    
from qiskit_ibm_runtime import QiskitRuntimeService

# Save an IBM Cloud account.
#in this case the token is created here: https://cloud.ibm.com/iam/apikeys
# and the instance here: https://cloud.ibm.com/quantum/instances
QiskitRuntimeService.save_account(channel="ibm_cloud",
                                  token="<YOUR_IBM_CLOUD_API_TOKEN>",
                                  instance="<YOUR_IBM_CLOUD_QUANTUM_INSTANCE_CRN>",
                                 overwrite=True)


## Session

In [6]:
from qiskit_ibm_runtime import Session, Estimator

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")

with Session(service=service, backend=backend, max_time="1h"): 
    estimator = Estimator()

    result = estimator.run(circuit, observable, theta).result()
    print(f">>> Expectation value from the first run: {result.values[0]}")

    result = estimator.run(circuit, observable, theta).result()
    print(f">>> Expectation value from the second run: {result.values[0]}")

>>> Expectation value from the first run: 1.5465


>>> Expectation value from the second run: 1.571


## Sampler

In [4]:
from qiskit.test.reference_circuits import ReferenceCircuits
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")

bell = ReferenceCircuits.bell()

with Session(service=service, backend=backend) as session:
    sampler = Sampler(session=session)

    job = sampler.run(bell, shots=1024)
    print(f"Job ID: {job.job_id()}")
    print(f"Job result: {job.result()}")

Job ID: cljf6s2jcm8ipr4e1320
Job result: SamplerResult(quasi_dists=[{3: 0.490234375, 0: 0.509765625}], metadata=[{'shots': 1024, 'circuit_metadata': {}}])


## Error supression

In [7]:
from qiskit_ibm_runtime import Options, Estimator

options = Options()
options.optimization_level = 3 # Error supression + advanced trasnpilation
options.execution.shots = 2048

In [8]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")
with Session(service=service, backend=backend) as session:
    estimator = Estimator(session=session, options=options)
    result = estimator.run(circuit, observable, theta).result()
    print(f">>> Metadata: {result.metadata[0]}")

>>> Metadata: {'variance': 9.09280014038086, 'shots': 2048}


## Error mittigation

In [9]:
from qiskit_ibm_runtime import Options

options = Options()
options.resilience_level = 1 # Minimal mitigation cost
options.execution.shots = 2048

In [11]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")
with Session(service=service, backend=backend) as session:
    estimator = Estimator(session=session, options=options)
    result = estimator.run(circuit, observable, theta).result()
    print(f">>> Metadata: {result.metadata[0]}")

>>> Metadata: {'variance': 9.014254570007324, 'shots': 2048, 'readout_mitigation_num_twirled_circuits': 16, 'readout_mitigation_shots_calibration': 8192}


<span style="font-size:10pt; font-weight:bold;"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an “existing” IBM asset as part of the IBM Quantum Accelerator contract. </span>
